# Simple Integer Algebra

In [1]:
import metadsl

In [2]:
class Integer(metadsl.Instance):
    @metadsl.call(lambda self, other: Integer)
    def __add__(self, other: "Integer") -> "Integer":
        ...
    
    @metadsl.call(lambda self, other: Integer)
    def __mul__(self, other):
        ...

@metadsl.call(lambda x: Integer)
def from_int(x: int) -> Integer:
    ...

In [3]:
metadsl.to_expression(from_int(1) +  from_int(1))

RecursiveCall(__add__, (RecursiveCall(from_int, (1,)), RecursiveCall(from_int, (1,))))

In [4]:
x = from_int(1) + from_int(2)
x

Integer(__call__=Call(__add__, (Integer(__call__=Call(from_int, (1,))), Integer(__call__=Call(from_int, (2,))))))

In [11]:
default_replacements = metadsl.Replacements()

@default_replacements.register(Integer)
def add_zero(x: Integer):
    return x + from_int(0), lambda: x

@default_replacements.register(Integer)
def mult_one(x: Integer):
    return x * from_int(1), lambda: x


In [12]:
int_replacements = metadsl.Replacements()
replacements = metadsl.ReplacementsSequence(default_replacements, int_replacements)

@int_replacements.register(None, None)
def add(x: int, y: int):
    return from_int(x) + from_int(y), lambda: from_int(x + y)

@int_replacements.register(None, None)
def mult(x: int, y: int):
    return from_int(x) * from_int(y), lambda: from_int(x * y)



@metadsl.call(lambda x: replacements)
def to_int(x: Integer) -> int:
    ...


@int_replacements.register_pure(None)
def _to_int(x: int):
    return to_int(from_int(x)), x

In [13]:
to_int(from_int(1) * from_int(100))

100

We see that the simplification rules take place here:

In [14]:
to_int(from_int("WRONG") + from_int(0))

'WRONG'